In [1]:
import pandas as pd
import numpy as np

In [2]:
data_dir = 'Data/cali_pred_data_2010_monthly/'

In [3]:
models = ['AV', 'GS', 'GM']

In [4]:
av_jan = pd.read_csv(data_dir + 'AV_clean_20101_align.csv')

In [60]:
#this creates different sized splits

num_lon_splits = 6
total_coords = 0
lon_splits = np.array_split(np.unique(np.sort(av_jan.lon.values)), num_lon_splits)
num_subset = {'AV' : 1, 'GS' : 1, 'GM' : 1}

for model in models:
    model_df = pd.read_csv('{}{}_clean_20101_align.csv'.format(data_dir,model))
    for i in range(num_lon_splits):
        sub_lon = lon_splits[i]
        df_sub = model_df.loc[model_df.lon.isin(sub_lon)]
        num_lat_splits = 5
        lat_splits = np.array_split(np.unique(np.sort(df_sub.lat.values)), num_lat_splits)
        for j in range(num_lat_splits):
            sub_lat = lat_splits[j]
            df_sub_sub = df_sub.loc[df_sub.lat.isin(sub_lat)]
            total_coords += df_sub_sub.shape[0]
            new_file = '{}{}_clean_20101_align.{}.csv'.format(data_dir, model, num_subset[model])
            print (new_file)
            num_subset[model] += 1
            df_sub_sub.to_csv(new_file, index = False)

In [83]:
# using coordinates from R
num_coords = 0
num_r_subsegs = 36
num_subsegs = {'AV' : 0, 'GS' : 0, 'GM' : 0}

for model in models:
    model_df = pd.read_csv('{}{}_clean_20101_align.csv'.format(data_dir,model))
    for i in range(1, num_r_subsegs + 1):
        file_name = '{}coordinates_{}.csv'.format(data_dir, i)
        coordinates = pd.read_csv(file_name)
        lon_list = list(np.around(np.array(np.unique(coordinates.x.values)),3))
        lat_list = list(np.around(np.array(np.unique(coordinates.y.values)),3))
        df_sub = model_df.loc[(model_df.lon.isin(lon_list)) & (model_df.lat.isin(lat_list))]
        if (df_sub.shape[0] != 0):
            df_sub = df_sub[['lon', 'lat', 'pm25']]
            num_subsegs[model] += 1
            new_file = '{}{}_clean_20101_align.{}.csv'.format(data_dir, model, num_subsegs[model])
            df_sub.to_csv(new_file, index = False)